In [1]:
import data
import model
import numpy as np
from random import choice, seed
from model import build_model, dense, dropout, prelu, build_sequential_model

Using Theano backend.


In [2]:
X, y, _, _, _ = data.load("~/Documents/flavours-of-physics-start/input/training.csv",
                          shuffle=True)
Xa, ya, wa, _, _ = data.load("~/Documents/flavours-of-physics-start/input/check_agreement.csv",
                             shuffle=False, weight=True)
Xa_train, ya_train, _, _, _ = data.load("~/Documents/flavours-of-physics-start/input/check_agreement.csv",
                             shuffle=True)
Xc, _, _, mc, _ = data.load("~/Documents/flavours-of-physics-start/input/check_correlation.csv",
                            shuffle=False, mass=True, test=True)

X, scaler = data.preprocess_data(X)
Xa, _ = data.preprocess_data(Xa, scaler)
Xa_train, _ = data.preprocess_data(Xa_train, scaler)
Xc, _ = data.preprocess_data(Xc, scaler)

X_test, _, _, _, ids = data.load("~/Documents/flavours-of-physics-start/input/test.csv", test=True,
                                 ids=True)
X_test, _ = data.preprocess_data(X_test, scaler)
print("Done")

Done


In [5]:
def connect(model1, model2):
    from keras.models import Sequential
    model = Sequential()
    model.add(model1)
    model.add(model2)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return model

def attach_grl(m, l):
    grl = model.GradientReversal(l)
    grl.set_input_shape(m.input_shape)
    return connect(grl, m)
    
def feature_extractor(input_size, output_size):
    model = build_sequential_model(input_size,
                                   [dense(150), prelu(), dropout(0.4),
                                    dense(140), prelu(), dropout(0.35),
                                    dense(130), prelu(), dropout(0.3),
                                    dense(120), prelu(), dropout(0.27)],
                                   output_size)
    return model


def label_classifier(input_size):
    clf = build_sequential_model(input_size,
                                 [dense(100), prelu(), dropout(0.3),
                                  dense(80), prelu(), dropout(0.25),
                                  dense(70), prelu(), dropout(0.2)], 2)
    return clf


def domain_classifier(input_size, l):
    clf = label_classifier(input_size)
    return attach_grl(clf, l)


def create(f, l, d, lam):
    import keras.models
    m = keras.models.Graph()
    m.add_input('input', input_shape=(f.input_shape[1],))
    m.add_node(f, name='feature_extractor', input='input')
    m.add_node(l, name='label_classifier', input='feature_extractor')
    grl = model.GradientReversal(lam)
    grl.set_input_shape(d.input_shape)
    m.add_node(grl, name='grl', input='feature_extractor')
    m.add_node(d, name='domain_classifier', input='grl')
    m.add_output('output', input='label_classifier')
    m.add_output('domain', input='domain_classifier')
    m.compile(loss={'output': 'categorical_crossentropy',
                    'domain': 'categorical_crossentropy'},
                  loss_weights={
                      'output': 1,
                      'domain': 1
                  },
                  optimizer='rmsprop')
    return m

In [ ]:
n_models = 2
n_epochs = 200
probs = None
for i in range(n_models):
    from keras.utils import np_utils
    domain_weight = 1
    np.random.seed(42)  # repeatability
    seed(42)

#     print("Model {}; l = {}; domain_weight = {}".format(i, lam, domain_weight))
    n_extracted_features = 60
    f = feature_extractor(X.shape[1], n_extracted_features)
    l = label_classifier(n_extracted_features)
    d = label_classifier(n_extracted_features)
    
    transfering_ratio = 0.7 # 0.1
    # Learning on train
    m = create(f, l, d, 0)
    f, l, d = model.fit_model(m, X, np_utils.to_categorical(y), np_utils.to_categorical(y), Xa, ya, wa, Xc, mc, X, y,
                    epoch_count=int((1 - transfering_ratio) * n_epochs),
                    batch_size=128, validation_split=0.05, verbose=2, show_metrics=True)
    # Transfering to check_agreement
    ya_output = model.predict_model(m, np.array(Xa_train))
    steps = 20
    for step in range(steps):
        lam = np.linspace(0.2, 10, steps)[step] # np.random.choice(np.linspace(1, 10, 10))
        print('lambda = ', lam)
        m = create(f, l, d, lam)
        f, l, d = model.fit_model(m, Xa_train, ya_output, np_utils.to_categorical(ya_train), Xa, ya, wa, Xc, mc, X, y,
                        epoch_count=int(transfering_ratio * n_epochs / steps),
                        batch_size=512, validation_split=0.5, verbose=2, show_metrics=True)
    
    # Output
    p = model.predict_probs_model(m, np.array(X_test))
    probs = p if probs is None else p + probs
probs /= n_models

Train on 64175 samples, validate on 3378 samples
Epoch 1/60
KS: 0.12292146023167205 : 0.09 / CvM: 0.0011056612980413886 : 0.002 / AUC: 0.9891548581634108
13s - loss: 0.8109 - val_loss: 0.6107
Epoch 2/60
KS: 0.14998764462843828 : 0.09 / CvM: 0.0010935212959419735 : 0.002 / AUC: 0.9910637329592038
12s - loss: 0.6302 - val_loss: 0.5650
Epoch 3/60
KS: 0.1631847375986843 : 0.09 / CvM: 0.0011380037040655165 : 0.002 / AUC: 0.9920196008175
12s - loss: 0.6022 - val_loss: 0.5634
Epoch 4/60
KS: 0.17848346179854696 : 0.09 / CvM: 0.0011091004252481583 : 0.002 / AUC: 0.9925532944680813
12s - loss: 0.5879 - val_loss: 0.5482
Epoch 5/60
KS: 0.17425993810475177 : 0.09 / CvM: 0.0011271506385750218 : 0.002 / AUC: 0.9928136932454933
12s - loss: 0.5746 - val_loss: 0.5682
Epoch 6/60
KS: 0.18412387228060118 : 0.09 / CvM: 0.0011508995791003152 : 0.002 / AUC: 0.9932001762504126
12s - loss: 0.5732 - val_loss: 0.5506
Epoch 7/60
KS: 0.18291293225526373 : 0.09 / CvM: 0.001132500662366561 : 0.002 / AUC: 0.9932804750

In [10]:
data.save_submission(ids, probs, "grl_prediction.csv")
!zip -9 -r grl_prediciton.csv.zip grl_prediction.csv

updating: grl_prediction.csv (deflated 53%)
